# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096847


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:05<01:22,  2.95s/it]

Rendering models:  10%|█         | 3/30 [00:06<00:59,  2.22s/it]

Rendering models:  13%|█▎        | 4/30 [00:06<00:41,  1.61s/it]

Rendering models:  17%|█▋        | 5/30 [00:08<00:44,  1.80s/it]

...

Rendering models:  87%|████████▋ | 26/30 [00:16<00:01,  2.15it/s]

Rendering models:  93%|█████████▎| 28/30 [00:17<00:00,  2.11it/s]

Rendering models:  97%|█████████▋| 29/30 [00:18<00:00,  2.45it/s]

Rendering models: 100%|██████████| 30/30 [00:18<00:00,  2.42it/s]

OregonHiker                           0.000894
Terrance_L._Johnson                   0.000664
ellivc                                0.000530
jnarayanbvg                           0.000535
not-logged-in-c6917d9b19f41e6c2995    0.001169
awright5                              0.001801
thiagoxxxx                            0.000500
jaksonA                               0.000639
izzyathogwarts                        0.023825
not-logged-in-509c39d7e75faf680e08    0.019610
not-logged-in-6ead6661c7dcef165774    0.000835
koungeo8                              0.001120
not-logged-in-54fd05425f46bbb14df6    0.008664
shocko61                              0.001343
not-logged-in-787639769ee932e26232    0.000853
crush202020                           0.000476
AlexYoung35                           0.000572
gabrielweiss                          0.001250
peakscience7                          0.000495
Lavadude                              0.030614
not-logged-in-90294de164951e098d0b    0.001301
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())